In [1]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, DateType
import pyspark as ps
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as stats
import re
import dateparser
import datetime

pd.set_option('display.max_columns', None)


In [2]:
current_df = pd.read_csv('~/dsi/capstones/cap_3/data/gsm.csv', sep=',')

# current_df

/Users/cv/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (75,76,77,79,80,81,82,83,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
cop_df = current_df.copy()

# Different Date Formats

# 0:     2009  February. Released 2009  April
# 23:    2011  February
# 45:    2012  Q3
# 103:   1997
# 144:   2003 1Q
# 202:   2007. Released 2007
# 459:   2Q  2003
# 461:   Exp. announcement 2012  August
# 1091:  Not officially announced yet 
# 1097:  (null) NaN
# 1649:  Not officially announced yet. Released Exp. release 2009  Q4
# 1703:  Feb-01
# 1708:  Q2 2001 ?
# 2228:  " "2013  February
# 2453:  Exp. announcement 2015  Q3
# 2987:  Not announced yet
# 5032:  2004 1 Q
# 5177:  2002  Oct
# 7090:  Never
# 7379:  4Q 2001

toy_df = cop_df.iloc[[0, 23, 45, 103, 144, 202, 459, 461, 1091, 1097, 1649,
                  1703, 1708, 2228, 2453, 2987, 5032, 5177, 7090, 7379]]

toy_df

,oem,model,network_technology,network_2g_bands,network_gprs,network_edge,launch_announced,launch_status,body_dimensions,body_weight,body_sim,body,display_type,display_size,display_resolution,platform_os,platform_chipset,platform_cpu,memory_card_slot,memory_internal,main_camera_single,main_camera_video,selfie_camera_single,selfie_camera_video,sound_loudspeaker,sound_3.5mm_jack,comms_wlan,comms_bluetooth,comms_gps,comms_radio,comms_usb,features_sensors,battery,battery_talk_time,misc_colors,misc_price,network_3g_bands,network_4g_bands,network,network_speed,platform_gpu,main_camera_features,sound,features,selfie_camera_features,battery_stand-by,tests_performance,tests_display,tests_camera,tests_loudspeaker,tests_audio_quality,tests_battery_life,display_protection,battery_charging,misc_models,comms_nfc,misc_sar_eu,battery_music_play,selfie_camera,display,misc_sar,features_browser,body_keyboard,main_camera_dual,comms_infrared_port,selfie_camera_dual,body_build,camera,memory_phonebook,memory_call_records,features_messaging,features_games,features_java,memory,sound_alert_types,features_clock,features_alarm,features_languages,main_camera_triple,selfie_camera_v1,main_camera,main_camera_quad,selfie_camera_triple,main_camera_v1,main_camera_dual_or_triple,main_camera_five
0,Acer,Chromebook Tab 10,No cellular connectivity,NaN,No,No,"2018, March","Available. Released 2018, July",238.3 x 172.2 x 9.9 mm (9.38 x 6.78 x 0.39 in),544.3 g (1.20 lb),No,Stylus,"IPS LCD capacitive touchscreen, 16M colors","9.7 inches, 291.4 cm (~71.0% screen-to-body ra...","1536 x 2048 pixels, 4:3 ratio (~264 ppi density)",Chrome OS,Rockchip RK3399,Hexa-core (4x Cortex-A53 & 2x Cortex-A72),"microSD, up to 128 GB","32 GB, 4 GB RAM",5 MP,720p,2 MP,NaN,"Yes, with stereo speakers",Yes,"Wi-Fi 802.11 b/g/n/ac, dual-band, WiFi Direct","4.1, A2DP, LE",NaN,No,"3.1, Type-C 1.0 reversible connector",Accelerometer,Non-removable Li-Po 4500 mAh battery (34 Wh),Up to 9 h (multimedia),"Black, Blue",About 330 EUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,Acer,Liquid Z220,GSM / HSPA,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,"2015, March","Available. Released 2015, April",125.3 x 64 x 9.6 mm (4.93 x 2.52 x 0.38 in),120 g (4.23 oz),Single SIM (Mini-SIM) or Dual SIM (Mini-SIM/Mi...,NaN,Capacitive touchscreen,"4.0 inches, 45.5 cm (~56.8% screen-to-body ratio)","480 x 800 pixels, 5:3 ratio (~233 ppi density)",Android 5.0 (Lollipop),Qualcomm Snapdragon 200 (28 nm),Dual-core 1.2 GHz Cortex-A7,"microSD, up to 32 GB (dedicated slot)","8 GB, 1 GB RAM","5 MP, AF",720p@30fps,2 MP,NaN,Yes,Yes,"Wi-Fi 802.11 b/g/n, hotspot","4.0, A2DP","Yes, with A-GPS",FM radio,microUSB 2.0,Accelerometer,Removable Li-Ion 1300 mAh battery,NaN,"Black, White",About 90 EUR,HSDPA 900 / 2100,NaN,HSDPA 850 / 1700(AWS),HSPA,Adreno 302,LED flash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,Acer,Liquid Z3,GSM / HSPA,GSM 900 / 1800 / 1900,Yes,Yes,"2013, August. Released 2013, September",Discontinued,109 x 60 x 10.4 mm (4.29 x 2.36 x 0.41 in),120 g (4.23 oz),"Single SIM (Micro-SIM) or Dual SIM (Micro-SIM,...",NaN,"TFT capacitive touchscreen, 16M colors","3.5 inches, 36.5 cm (~55.8% screen-to-body ratio)","320 x 480 pixels, 3:2 ratio (~165 ppi density)",Android 4.2 (Jelly Bean),Mediatek MT6572 (28 nm),Dual-core 1.0 GHz Cortex-A7,"microSD, up to 32 GB (dedicated slot)","4 GB, 512 MB RAM",3.15 MP,Yes,NaN,NaN,Yes,Yes,"Wi-Fi 802.11 a/b/g/n, hotspot","3.0, A2DP, EDR","Yes, with A-GPS",Stereo FM radio,microUSB 2.0,"Accelerometer, proximity",Removable Li-Ion 1500 mAh battery,NaN,"Rock Black, Classic White",About 100 EUR,HSDPA 900 / 2100,NaN,NaN,HSPA 21.1/5.76 Mbps,Mali-400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [4]:
# DATE CONVERSION

# Parsing variables
years = range(1994, 2017)

months = ['January', 'February', 'March', 'April', 'May', 'June', 
         'July', 'August', 'September', 'October', 'November', 'December']

months_short = ['Jan-', 'Feb-', 'Mar-', 'Apr-', 'May-', 'Jun-', 
                'Jul-', 'Aug-', 'Sep-', 'Oct-', 'Nov-', 'Dec-']

months_num = np.linspace(0.0, 1.0, num=12)

months_int = range(1, 12)

months_dict = {
    'January':1, 
    'February':2, 
    'March':3, 
    'April':4, 
    'May':5, 
    'June':6, 
    'July':7, 
    'August':8, 
    'September':9, 
    'October':10, 
    'November':11, 
    'December':12
}

quarters = ['Q1', '1Q', 'Q2', '2Q', 'Q3', '3Q', 'Q4', '4Q']

quarters_dict = {
  'Q1': 'February',
  '1Q': 'February',
  'Q2': 'May',
  '2Q': 'May',
  '3Q': 'August',
  'Q3': 'August',
  '4Q': 'November',
  'Q4': 'November'
}


In [5]:
def parse_date(date_str):
    
    # Removes preceding and proceding spaces
    date_str = date_str.strip()
    
    # Changes strings with "?" to "remove"
    if '?' in date_str:
        date_str = 'remove'
    
    # Removes errant release dates
    if '.' in date_str:
        split_str = date_str.split('.')
        date_str = split_str[0]
    
    # Changes Quarter to Middle Month of Quarter
    for key, val in quarters_dict.items():
        if key in date_str:
            split_2 = date_str.split()
            split_2[split_2.index(key)] = val
            date_str = " ".join(split_2)

    # Converts Year Only to Year + ' July'
    if len(date_str) < 5:
        date_str += ' July'    
    
    # If date is formated in short form i.e. "Feb-01"
    m_index = 0
    for m in months_short:
        if m in date_str:
            split_3 = date_str.split('-')
            split_3[0] = months[m_index]
            split_3[1] = f'20{split_3[1]}'
            date_str = " ".join(split_3)
        m_index += 1
       
    # Date Parsing str using module dataparser    
    date_tm = dateparser.parse(date_str)
    if isinstance(date_tm, datetime.date):
        date_tm = date_tm - datetime.timedelta(days=date_tm.day-1)
        
        
    return date_tm



# parse_date('2009  February. Released 2009  April')
# parse_date('2009  Q1. Released 2009  April')
# parse_date('2012  Q3')
# parse_date('1Q 2003')
# parse_date('2003')
# parse_date('Feb-01')


In [6]:
# Re-sort date str - Maybe Delete
def re_sort(date_str):
    sorted_str = ''
    
    # Re-sort Month and Year
    for m in months:
#         print(m)
        if m in date_str:
            sorted_str = m
    for y in years:
#         print(y)
        if str(y) in date_str:
            sorted_str = sorted_str + ' ' + str(y)
            break
            
    return sorted_str

# date_test = '2009  February. Released 2009  April'
# re_sort(parse_date(date_test))


In [7]:
# Is Unique Function - Delete
def is_unique(check_list):
    unique_set = set(check_list)
    unique_list = list(unique_set)
    return unique_list

# toy_list = toy_df['brand'].tolist()
# print(toy_list)

# u_list = is_unique(toy_list)
# u_list.sort()
# u_list



In [8]:
# Find Model Names
def unique_models(_list, split_1 = None, split_2 = None):
#     _list = model_list
    unique_list = []
    
    if split_1 is None and split_2 is None:
        for n in _list:
            unique_list.append(n)
            unique_list = is_unique(unique_list)
            
            
#     elif split_1 or split_2:
#         for n in _list:
#             n_dict = {}
#             split_atr = n.split(split_2)
#             split_atr = split_atr.split(split_1)
#             n_dict[split_atr[0]] = []
#             comb_back = ' '.join(split_atr[1:])
#             n_dict[split_atr[0]].extend(comb_back)
#             unique_list.append(n_dict)
    
    elif split_1:
        for n in _list:
            n_dict = {}
            split_atr = n.split(split_1)
            n_dict[split_atr[0]] = []
            n_dict[split_atr[0]].extend(split_atr[1:])
            unique_list.append(n_dict)
    
    return unique_list

toy_list = toy_df['model'].tolist()
u_brands = unique_models(toy_list, ' ')
u_brands


[{'Chromebook': ['Tab', '10']},
 {'Liquid': ['Z220']},
 {'Liquid': ['Z3']},
 {'5': []},
 {'Fierce': ['XL']},
 {'Fire': ['E']},
 {'P8': ['Pro']},
 {'X4': ['Soul', 'Infinity', 'L']},
 {'Torch': ['9800']},
 {'Storm2': ['9550']},
 {'C550': []},
 {'Torino': []},
 {'Note': ['3', 'Lite']},
 {'One': ['Dual', 'Sim']},
 {'P10': []},
 {'Iris': ['Fuel', '60']},
 {'1650': []},
 {'8855': []},
 {'M300': []},
 {'CXT65': []}]

In [9]:
# Remove Date-less Objects
def refactor_df(df, col='launch_announced'):
    step_df = df[df['launch_announced'].notna()]
    
    step_df['launch_announced'] = step_df['launch_announced'].apply(parse_date)
    
    step_df['launch_announced'] = step_df['launch_announced'].dt.to_period('M')
    
    return step_df


In [10]:
# Cast NaN/Null

make_nan = ['No', 'N/A', 'No cellular connectivity']

def cast_nan(step_df, make_nan):
#     if nan_str in nan_list:
#         nan_str = NaN
    df_obj = step_df.select_dtypes(['object'])

    step_df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

    step_df.replace(to_replace=make_nan, value=np.nan, inplace=True)
    
    return step_df


In [11]:
# Drop Columns



In [12]:
toy_df = refactor_df(toy_df)

toy_df = cast_nan(toy_df, make_nan)

# toy_df

In [13]:
toy_df

,oem,model,network_technology,network_2g_bands,network_gprs,network_edge,launch_announced,launch_status,body_dimensions,body_weight,body_sim,body,display_type,display_size,display_resolution,platform_os,platform_chipset,platform_cpu,memory_card_slot,memory_internal,main_camera_single,main_camera_video,selfie_camera_single,selfie_camera_video,sound_loudspeaker,sound_3.5mm_jack,comms_wlan,comms_bluetooth,comms_gps,comms_radio,comms_usb,features_sensors,battery,battery_talk_time,misc_colors,misc_price,network_3g_bands,network_4g_bands,network,network_speed,platform_gpu,main_camera_features,sound,features,selfie_camera_features,battery_stand-by,tests_performance,tests_display,tests_camera,tests_loudspeaker,tests_audio_quality,tests_battery_life,display_protection,battery_charging,misc_models,comms_nfc,misc_sar_eu,battery_music_play,selfie_camera,display,misc_sar,features_browser,body_keyboard,main_camera_dual,comms_infrared_port,selfie_camera_dual,body_build,camera,memory_phonebook,memory_call_records,features_messaging,features_games,features_java,memory,sound_alert_types,features_clock,features_alarm,features_languages,main_camera_triple,selfie_camera_v1,main_camera,main_camera_quad,selfie_camera_triple,main_camera_v1,main_camera_dual_or_triple,main_camera_five
0,Acer,Chromebook Tab 10,NaN,NaN,NaN,NaN,2018-03,"Available. Released 2018, July",238.3 x 172.2 x 9.9 mm (9.38 x 6.78 x 0.39 in),544.3 g (1.20 lb),NaN,Stylus,"IPS LCD capacitive touchscreen, 16M colors","9.7 inches, 291.4 cm (~71.0% screen-to-body ra...","1536 x 2048 pixels, 4:3 ratio (~264 ppi density)",Chrome OS,Rockchip RK3399,Hexa-core (4x Cortex-A53 & 2x Cortex-A72),"microSD, up to 128 GB","32 GB, 4 GB RAM",5 MP,720p,2 MP,NaN,"Yes, with stereo speakers",Yes,"Wi-Fi 802.11 b/g/n/ac, dual-band, WiFi Direct","4.1, A2DP, LE",NaN,NaN,"3.1, Type-C 1.0 reversible connector",Accelerometer,Non-removable Li-Po 4500 mAh battery (34 Wh),Up to 9 h (multimedia),"Black, Blue",About 330 EUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,Acer,Liquid Z220,GSM / HSPA,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,2015-03,"Available. Released 2015, April",125.3 x 64 x 9.6 mm (4.93 x 2.52 x 0.38 in),120 g (4.23 oz),Single SIM (Mini-SIM) or Dual SIM (Mini-SIM/Mi...,NaN,Capacitive touchscreen,"4.0 inches, 45.5 cm (~56.8% screen-to-body ratio)","480 x 800 pixels, 5:3 ratio (~233 ppi density)",Android 5.0 (Lollipop),Qualcomm Snapdragon 200 (28 nm),Dual-core 1.2 GHz Cortex-A7,"microSD, up to 32 GB (dedicated slot)","8 GB, 1 GB RAM","5 MP, AF",720p@30fps,2 MP,NaN,Yes,Yes,"Wi-Fi 802.11 b/g/n, hotspot","4.0, A2DP","Yes, with A-GPS",FM radio,microUSB 2.0,Accelerometer,Removable Li-Ion 1300 mAh battery,NaN,"Black, White",About 90 EUR,HSDPA 900 / 2100,NaN,HSDPA 850 / 1700(AWS),HSPA,Adreno 302,LED flash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,Acer,Liquid Z3,GSM / HSPA,GSM 900 / 1800 / 1900,Yes,Yes,2013-08,Discontinued,109 x 60 x 10.4 mm (4.29 x 2.36 x 0.41 in),120 g (4.23 oz),"Single SIM (Micro-SIM) or Dual SIM (Micro-SIM,...",NaN,"TFT capacitive touchscreen, 16M colors","3.5 inches, 36.5 cm (~55.8% screen-to-body ratio)","320 x 480 pixels, 3:2 ratio (~165 ppi density)",Android 4.2 (Jelly Bean),Mediatek MT6572 (28 nm),Dual-core 1.0 GHz Cortex-A7,"microSD, up to 32 GB (dedicated slot)","4 GB, 512 MB RAM",3.15 MP,Yes,NaN,NaN,Yes,Yes,"Wi-Fi 802.11 a/b/g/n, hotspot","3.0, A2DP, EDR","Yes, with A-GPS",Stereo FM radio,microUSB 2.0,"Accelerometer, proximity",Removable Li-Ion 1500 mAh battery,NaN,"Rock Black, Classic White",About 100 EUR,HSDPA 900 / 2100,NaN,NaN,HSPA 21.1/5.76 Mbps,Mali-400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [14]:
period_df = refactor_df(cop_df)

period_df = cast_nan(period_df, make_nan)


In [15]:
period_df.head(10)

,oem,model,network_technology,network_2g_bands,network_gprs,network_edge,launch_announced,launch_status,body_dimensions,body_weight,body_sim,body,display_type,display_size,display_resolution,platform_os,platform_chipset,platform_cpu,memory_card_slot,memory_internal,main_camera_single,main_camera_video,selfie_camera_single,selfie_camera_video,sound_loudspeaker,sound_3.5mm_jack,comms_wlan,comms_bluetooth,comms_gps,comms_radio,comms_usb,features_sensors,battery,battery_talk_time,misc_colors,misc_price,network_3g_bands,network_4g_bands,network,network_speed,platform_gpu,main_camera_features,sound,features,selfie_camera_features,battery_stand-by,tests_performance,tests_display,tests_camera,tests_loudspeaker,tests_audio_quality,tests_battery_life,display_protection,battery_charging,misc_models,comms_nfc,misc_sar_eu,battery_music_play,selfie_camera,display,misc_sar,features_browser,body_keyboard,main_camera_dual,comms_infrared_port,selfie_camera_dual,body_build,camera,memory_phonebook,memory_call_records,features_messaging,features_games,features_java,memory,sound_alert_types,features_clock,features_alarm,features_languages,main_camera_triple,selfie_camera_v1,main_camera,main_camera_quad,selfie_camera_triple,main_camera_v1,main_camera_dual_or_triple,main_camera_five
0,Acer,Chromebook Tab 10,NaN,NaN,NaN,NaN,2018-03,"Available. Released 2018, July",238.3 x 172.2 x 9.9 mm (9.38 x 6.78 x 0.39 in),544.3 g (1.20 lb),NaN,Stylus,"IPS LCD capacitive touchscreen, 16M colors","9.7 inches, 291.4 cm (~71.0% screen-to-body ra...","1536 x 2048 pixels, 4:3 ratio (~264 ppi density)",Chrome OS,Rockchip RK3399,Hexa-core (4x Cortex-A53 & 2x Cortex-A72),"microSD, up to 128 GB","32 GB, 4 GB RAM",5 MP,720p,2 MP,NaN,"Yes, with stereo speakers",Yes,"Wi-Fi 802.11 b/g/n/ac, dual-band, WiFi Direct","4.1, A2DP, LE",NaN,NaN,"3.1, Type-C 1.0 reversible connector",Accelerometer,Non-removable Li-Po 4500 mAh battery (34 Wh),Up to 9 h (multimedia),"Black, Blue",About 330 EUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Acer,Iconia Talk S,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2,Yes,Yes,2016-08,"Available. Released 2016, October",191.7 x 101 x 9.4 mm (7.55 x 3.98 x 0.37 in),260 g (9.17 oz),Dual SIM (Micro-SIM/Nano-SIM),NaN,"IPS LCD capacitive touchscreen, 16M colors","7.0 inches, 135.1 cm (~69.8% screen-to-body ra...","720 x 1280 pixels, 16:9 ratio (~210 ppi density)",Android 6.0 (Marshmallow),Mediatek MT8735,Quad-core 1.3 GHz Cortex-A53,"microSD, up to 128 GB (dedicated slot)","16/32 GB, 2 GB RAM","13 MP, AF",1080p@30fps,2 MP,720p,Yes,Yes,"Wi-Fi 802.11 a/b/g/n, Wi-Fi Direct, hotspot","4.0, A2DP","Yes, with A-GPS, GLONASS",FM radio,microUSB 2.0,"Accelerometer, proximity",Non-removable Li-Ion 3400 mAh battery (12.92 Wh),Up to 9 h (multimedia),Black,About 170 EUR,HSDPA 850 / 1900 / 2100,"LTE band 1(2100), 3(1800), 7(2600), 8(900), 20...","LTE band 3(1800), 7(2600), 8(900), 28(700)","HSPA 42.2/11.5 Mbps, LTE Cat4 150/50 Mbps",Mali-T720MP2,"HDR, panorama",DTS HD sound,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Acer,Liquid Z6 Plus,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,2016-08,"Available. Released 2016, December",153.8 x 75.6 x 8.5 mm (6.06 x 2.98 x 0.33 in),169 g (5.96 oz),Single SIM (Micro-SIM) or Hybrid Dual SIM (Mic...,NaN,"IPS LCD capacitive touchscreen, 16M colors","5.5 inches, 83.4 cm (~71.7% screen-to-body ratio)","1080 x 1920 pixels, 16:9 ratio (~401 ppi density)",Android 6.0 (Marshmallow),Mediatek MT6753 (28 nm),Octa-core 1.3 GHz Cortex-A53,"microSD, up to 256 GB (uses shared SIM slot)","32 GB, 3 GB RAM","13 MP, AF",1080p@30fps,5 MP,NaN,Yes,Yes,"Wi-Fi 802.11 b/g/n, Wi-Fi Direct, hotspot","4.0, A2DP","Yes, with A-GPS",FM radio,

In [62]:
# Screen Size

def screen_refactor(s, split_char=' '):
    s = str(s)
    s = s.replace('| -', '')
    s = s.replace('|', '')
    s = s.strip()
    
    
    if isinstance(s, str):
        split_s = s.split(split_char)
        try 
        return split_s[0]
    else:
        return s

# test_s = '1.6 inches (~25.2% screen-to-body ratio)'
# screen_refactor(test_s)


# def is_tablet(size_flt, output='tablet'):
#     if size_flt 
    
    

    


SyntaxError: invalid syntax (<ipython-input-62-28e998b432c1>, line 12)

In [63]:
period_df['screen_inches'] = period_df['display_size'].apply(screen_refactor)


NameError: name 'screen_refactor' is not defined

In [64]:
# period_df.agg({'launch_announced':'count', 'misc_price':'mean'})
fig = plt.figure(figsize=(12,4))

# toy_df.groupby('launch_announced').mean()['misc_price']

ax1 = period_df.groupby('launch_announced').mean()['misc_price'].plot(
        xlim=[pd.Timestamp('2005-08-01'), pd.Timestamp('2017-10-01')], ylim=[0, 750])

ax1.set_ylabel('Average Price (EUR)')
ax1.set_xlabel('Month Announced')
# ax1.title("Average Price (EUR) of Phones Announced on Given Month")


KeyError: 'misc_price'

<Figure size 864x288 with 0 Axes>

In [39]:
# period_df.agg({'launch_announced':'count', 'misc_price':'mean'})
fig2 = plt.figure(figsize=(12,4))

# toy_df.groupby('launch_announced').mean()['screen_inches']

ax2 = period_df.groupby('launch_announced').mean()['screen_inches'].plot(
        xlim=[pd.Timestamp('2005-08-01'), pd.Timestamp('2017-10-01')])

ax2.set_ylabel('Average Price (EUR)')
ax2.set_xlabel('Month Announced')
# ax1.title("Average Price (EUR) of Phones Announced on Given Month")

KeyError: 'screen_inches'

<Figure size 864x288 with 0 Axes>

In [40]:
# Group By

period_df.groupby('launch_announced').count()

,oem,model,network_technology,network_2g_bands,network_gprs,network_edge,launch_status,body_dimensions,body_weight,body_sim,body,display_type,display_size,display_resolution,platform_os,platform_chipset,platform_cpu,memory_card_slot,memory_internal,main_camera_single,main_camera_video,selfie_camera_single,selfie_camera_video,sound_loudspeaker,sound_3.5mm_jack,comms_wlan,comms_bluetooth,comms_gps,comms_radio,comms_usb,features_sensors,battery,battery_talk_time,misc_colors,misc_price,network_3g_bands,network_4g_bands,network,network_speed,platform_gpu,main_camera_features,sound,features,selfie_camera_features,battery_stand-by,tests_performance,tests_display,tests_camera,tests_loudspeaker,tests_audio_quality,tests_battery_life,display_protection,battery_charging,misc_models,comms_nfc,misc_sar_eu,battery_music_play,selfie_camera,display,misc_sar,features_browser,body_keyboard,main_camera_dual,comms_infrared_port,selfie_camera_dual,body_build,camera,memory_phonebook,memory_call_records,features_messaging,features_games,features_java,memory,sound_alert_types,features_clock,features_alarm,features_languages,main_camera_triple,selfie_camera_v1,main_camera,main_camera_quad,selfie_camera_triple,main_camera_v1,main_camera_dual_or_triple,main_camera_five
launch_announced,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1995-07,2,2,2,2,0,0,2,2,2,2,0,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,1,2,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,2,1,0,0,0,0,0,0,0,1,1,2,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0
1996-07,1,1,1,1,0,0,1,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0
1997-07,7,7,7,7,0,0,7,7,7,7,0,7,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,6,5,7,0,0,0,0,0,0,0,4,2,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,7,1,7,0,0,0,7,6,6,4,0,0,0,0,0,0,0,0
1998-07,23,23,23,23,0,0,23,23,23,23,0,23,0,23,0,0,1,0,2,0,0,0,0,1,0,0,0,0,0,0,23,11,11,23,0,0,0,0,0,0,0,5,18,0,11,0,0,0,0,0,0,0,0,0,0,1,0,0,16,1,1,0,0,6,0,0,0,23,22,23,6,0,2,23,20,19,13,0,0,0,0,0,0,0,0
1999-07,41,41,41,41,0,0,41,41,41,41,0,41,1,41,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,41,33,32,41,0,0,0,0,0,0,0,7,35,0,32,0,0,0,0,0,0,0,0,0,0,1,0,0,36,0,2,0,0,5,0,0,0,26,37,41,10,0,14,41,32,31,19,0,0,0,0,0,0,0,0
2000-07,59,59,59,59,0,0,59,59,59,59,1,59,1,59,1,0,1,1,1,0,0,0,0,12,0,0,0,0,3,0,59,47,47,59,0,0,0,0,0,0,0,21,56,0,47,0,0,0,0,0,0,0,0,0,0,2,0,0,45,0,29,0,0,13,0,0,0,44,53,58,37,0,25,59,56,51,30,0,0,0,0,0,0,0,0
2000-08,1,1,1,1,0,0,1,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1,1,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0
2001-02,1,1,1,1,0,0,1,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0
2001-05,1,1,1,1,0,0,1,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0


In [41]:
# Find Floating Point Regex



In [42]:
print(toy_df['launch_announced'].max())
print(toy_df['launch_announced'].min())


# def day_diff(a,b) = toy_df['launch_announced'].max() - toy_df['launch_announced'].min()
#     return a.max() - toy_df['launch_announced'].min()
def month_diff(a, b):
    return 12 * (a.year - b.year) + (a.month - b.month)

month_diff(toy_df['launch_announced'].max(), toy_df['launch_announced'].min())

2018-03
2002-02


193

In [44]:
df_feature = toy_df[['comms_bluetooth', 'launch_announced']]
    
date_max = df_feature['launch_announced'].max()
date_min = df_feature['launch_announced'].min()

month_period = month_diff(date_max, date_min)
day_period = date_max - date_min

# date_array = {date_min:0}
# for x in range(1, month_period):
#     date_n = date_array[x-1] + datetime.timedelta(days=31)
#     date_n = date_n - datetime.timedelta(days=date_n.day-1)
#     date_array.append({date_n: 0})
    
# plot.bar(date_array )  

# date_array2 = pd.date_range(date_min, date_max, freq='M')

    
date_counts = df_feature.groupby([df_feature['launch_announced'].dt.year, df_feature['launch_announced'].dt.month]).count()

date_counts['announced_year'] = date_counts.index.get_level_values(0).astype(int)
date_counts['announced_month'] = date_counts.index.get_level_values(1).astype(int)
date_counts = date_counts.reset_index(drop=True)
date_counts = date_counts.drop('launch_announced', axis=1)

# date_counts

date_counts['datetime'] = date_counts.apply(lambda row: datetime.date(year=row[1], month=row[2], day=1), axis=1)

# date_array = [date_min + datetime.timedelta(months=x) for x in range(0, (date_max-date_min)]
date_counts = date_counts.set_index('datetime')
date_counts.index = pd.to_datetime(date_counts.index)
date_counts.index = date_counts.index.to_period('M')

date_array2 = pd.DataFrame(index=date_array2.to_period('M'))
date_array2.join(date_counts)

AttributeError: Can only use .dt accessor with datetimelike values

In [45]:
# Histogram Plot of Datetimes

def period_hist(
    df_count = None, feature_count = None,
    start_date = (1994, 7, 0), end_date = (2017, 6, 0), format_tight = None,
    apple_date = None, year_only = None
):
    
    df_feature = df_count[[feature_count, 'launch_announced']]
    
    date_max = df_feature['launch_announced'].max()
    date_min = df_feature['launch_announced'].min()
    
    month_period = month_diff(date_max, date_min)
    day_period = date_max - date_min
    
    for x in range(0, month_period):
        
                   
    date_array = [date_min + datetime.timedelta(months=x) for x in range(0, (date_max-date_min)]
    
    if format_tight:
        date_diff = df_count['launch_announced'].max() - df_count['launch_announced'].min()
        
#     df_feature.launch_announced.dt.month.hist(bins=month_period)    
    
    date_tm = date_tm - datetime.timedelta(days=date_tm.day-1)
    
#     x_dates = df_feature[feature_count]
#     y_count = [1] * len(x_dates)
    
    date_counts = df_feature.groupby([df_feature['launch_announced'].dt.year, df_feature['launch_announced'].dt.month]).count()
    
    
    
#     df_feature.groupby([df_feature['launch_announced'].dt.year, df_feature['launch_announced'].dt.month]).count().barplot()
        
#     parse_date(str(df_count.launch_announced))
    
#     df_feature.hist(figsize=(12, 5), grid=False)
#     plt.tight_layout() 
    
#     df_feature.groupby([df_feature['launch_announced'].dt.year, df_feature['launch_announced'].dt.month]).count().plot(kind="bar") 
 
    return 

period_hist(df_count = toy_df, feature_count = 'weight_g')



IndentationError: expected an indented block (<ipython-input-45-e641ec3d06ab>, line 20)

In [207]:
# Additional Functions



